In [61]:
import pandas as pd
import numpy as np
df=pd.read_csv("Real estate/real_estate_train.csv")
df.head(5)

,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,2012.917,32.0,84.87882,NaN,24.98298,121.54024,37.9
1,2012.917,19.5,306.59470,9.0,24.98034,121.53951,42.2
2,2013.583,13.3,561.98450,5.0,24.98746,121.54391,47.3
3,2013.500,13.3,561.98450,5.0,24.98746,121.54391,54.8
4,2012.833,5.0,390.56840,5.0,24.97937,121.54245,43.1


In [62]:
def getLabelFeatures(df):
    feature_columns=["X1 transaction date", "X2 house age", "X3 distance to the nearest MRT station", "X4 number of convenience stores", "X5 latitude", "X6 longitude"]
    label_column="Y house price of unit area"
    features=df.loc[:,feature_columns]
    label=df.loc[:,label_column]
    return label, features
label, features = getLabelFeatures(df)

In [63]:
features.isnull().sum()

X1 transaction date                       1
X2 house age                              3
X3 distance to the nearest MRT station    0
X4 number of convenience stores           4
X5 latitude                               2
X6 longitude                              1
dtype: int64

In [68]:
X=features.values
y=label.values

# preprocessing data

from sklearn.impute import SimpleImputer

def fillNull(matrix):
    x1_impute=SimpleImputer(np.nan, strategy="mean")
    matrix[:,0:2]=x1_impute.fit_transform(matrix[:,0:2])

    x4_6_impute=SimpleImputer(np.nan, strategy="mean")
    matrix[:,3:6]=x4_6_impute.fit_transform(matrix[:,3:6])
    return matrix
    
X = fillNull(X)

new_df=pd.DataFrame.from_records(X, columns=feature_columns)
new_df.isnull().sum()

X1 transaction date                       0
X2 house age                              0
X3 distance to the nearest MRT station    0
X4 number of convenience stores           0
X5 latitude                               0
X6 longitude                              0
dtype: int64

In [66]:
# my shitty implementation
ones=np.ones((X.shape[0],1))
X_bar=np.concatenate((ones,X),axis=1)
inverse=np.linalg.pinv(np.asarray(np.dot(X_bar.T,X_bar), dtype="float"))
w=np.dot(np.dot(X_bar.T, y), inverse)
w # wtf, no ra toan nan :/

array([nan, nan, nan, nan, nan, nan, nan])

In [40]:
# when i use lib

from sklearn.linear_model import LinearRegression
linear=LinearRegression()

# start trainning the model
linear.fit(X,y)
print(linear.coef_)# without w0
print(linear.intercept_)# w0

# test trained result
from sklearn.metrics import mean_squared_error, mean_absolute_error

dfTest=pd.read_csv("Real estate/real_estate_test.csv")
testLabel, testFeatures = getLabelFeatures(dfTest)
X_test = testFeatures.values
y_test = testLabel.values
X_test = fillNull(X_test)

predict_test=linear.predict(X_test)

print("root mean squared error on test: {}".format(np.sqrt(mean_squared_error(y_test, predict_test))))

[ 5.84547031e+00 -1.82925032e-01 -6.08677337e-03  8.93757524e-01
  1.07943416e+02 -4.29868371e+01]
-9194.375151406071
root mean squared error on test: 7.496055380340487
